In [3]:
import nltk
from nltk import FreqDist
from nltk.collocations import *
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import gensim
from gensim.models.word2vec import Word2Vec
from sklearn.manifold import TSNE
from bokeh.io import output_notebook
from bokeh.plotting import show, figure
from textblob import TextBlob
%matplotlib inline

In [2]:
path = "C:\\Users\\ziadm\\Downloads\\Web Data mining\\RAP\\combined\\"
prefix = os.listdir(path)
print(prefix)

def remove_utf(text):
    return re.sub(r'[^\x00-\x7f]',r' ',text)



['Week1_Q1.txt', 'Week1_Q2.txt', 'Week1_Q3.txt', 'Week1_Q4.txt', 'Week2_Q1.txt', 'Week2_Q2.txt', 'Week2_Q3.txt']


In [3]:
#Read the dataset
dataset={} #nltk text from tokens
dataset_raw = {} 
allFeatures=set()
tot_articles = 0
articles_count={}
raw_corpus = {} #used in sumerization
dataset2= set()
dataset3=[]
N={} # Number of articles in each corpus

for i,_ in enumerate(prefix):
    fileName = path + prefix[i]
    f=open(fileName,'r',encoding="utf8")
    text = ''
    text_raw = '' 
    
    lines = f.readlines()
    #print(_,'OK') #load test
    tot_articles+=len(lines)
    articles_count[str(_)] = len(lines)
    dataset_raw[str(_)] = list(map(lambda line: line.lower(), lines))

    for line in lines:
        dataset2.add(line.lower())
        dataset3.append(remove_utf(line.lower()))
        text+=line.replace('\n',' ').lower()
        text_raw = line.lower()
    f.close
    N[str(_)]=len(lines)
    
    tokens = nltk.word_tokenize(text)
    dataset[str(_)] = nltk.Text(tokens)
    raw_corpus[_] = text


Functions for normalization

In [4]:
#Normalization
def remove_punctuation(corpus):
    punctuations = ".,\"-\\/#!?$%\^&\*;:{}=\-_'~()"    
    filtered_corpus = [token for token in corpus if (not token in punctuations)]
    return filtered_corpus

def apply_stopwording(corpus, min_len):
    filtered_corpus = [token for token in corpus if (not token in stopwords.words('english') and len(token)>min_len)]
    return filtered_corpus

def removeAbb(x):
    lst = {'Dx':'diagnosed' ,
           'Rx':'prescription',
           'OTC':'Over The Counter',
           'DFL':'Drug Fact Label',
           'AUT':'Application Under Test'}
    for i in x:
        if i in lst:
            i = lst[i]
    return x

def apply_lemmatization(corpus):
    lemmatizer = nltk.WordNetLemmatizer()
    normalized_corpus = [lemmatizer.lemmatize(token) for token in corpus]
    return normalized_corpus

Removing Punctuations & Stopwording

In [5]:
#Preprocessing
dataset_clean={} #dict of tokens

for i in dataset:
    #print ('Processing %s' % str(i))
    dataset_clean[i] = apply_lemmatization(removeAbb(apply_stopwording(remove_punctuation(dataset[i]), 3)))
    #print (dataset_clean[i])


In [6]:
dataset_clean.keys()

dict_keys(['Week1_Q1.txt', 'Week1_Q2.txt', 'Week1_Q3.txt', 'Week1_Q4.txt', 'Week2_Q1.txt', 'Week2_Q2.txt', 'Week2_Q3.txt'])

In [7]:
#create a nltk.Text dict with clened dataset
clean_text = {}
for i in dataset_clean:
    clean_text[i] = nltk.Text(dataset_clean[i])

In [8]:
print(clean_text)

{'Week1_Q1.txt': <Text: dont think patient seem able make correct decision...>, 'Week1_Q2.txt': <Text: likelihood think demographic detract standpoint want treatment think...>, 'Week1_Q3.txt': <Text: biased indication short answer agency like safety check...>, 'Week1_Q4.txt': <Text: would anticipate rate similar untreated male population 1000...>, 'Week2_Q1.txt': <Text: would suggest label enlarged prostate urinary symptom reliever...>, 'Week2_Q2.txt': <Text: appropriate self-identification symptom confusion sexual dysfucntion inappropriate expectation...>, 'Week2_Q3.txt': <Text: look pretty good consider adding heart failure cause...>}


In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ziadm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Collocations

Interesting terms that appear ogether in documents (collocations)

In [10]:
for i in clean_text:
    print()
    print(i)
    print()
    print(clean_text[i].collocations())


Week1_Q1.txt

alpha blocker; compare proposed; actual study; practice compare;
typical practice; regard risk; tamsulosin hydrochloride; population
regard; population typical; advisory committee; standard care;
proposed population; adverse event; exploratory actual; risk otcs;
long period; period time; trying treat; thinking much; without
thinking
None

Week1_Q2.txt

prostate cancer; tamsulosin hydrochloride; bladder cancer; medically
significant; safety algorithm; benign prostatic; bother getting;
bph/luts obtain; checked discus; comfortable enough; getting checked;
obtain partial; partial relief; prostatic hyperplasia; tail
distribution; concerned concomitant; confidence interval; full
evaluation; importance including; plus incident
None

Week1_Q3.txt

called medically; come member; complication process; medically
relevant; member disagree; yield called; unnecessary complication;
limited yield; process limited; condition concern; relevant condition;
product correctly; might come; uri

# Summarization of documents

In [11]:
#these funcions use the raw_corpus

def getCollocations(text, min_freq):
    bigrams = nltk.collocations.BigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(text)
    finder.apply_freq_filter(min_freq)
    collocations = finder.nbest(bigrams.pmi, 20)
    return collocations

def getSummary(article, min_freq):
    sentences = sent_tokenize(article)
    #tokens = [word_tokenize(sentence.lower()) for sentence in sentences]
    tokens = nltk.word_tokenize(article.lower())
    doc = nltk.Text(tokens)
    doc_clean = nltk.Text(apply_lemmatization(removeAbb(apply_stopwording(remove_punctuation(doc), 3))))
    collocations = getCollocations(doc_clean,min_freq)
    #print('Collocations: ', collocations)
    #print()
    
    summary = []
    for c in collocations:
        for sentence in sentences:
            term1 = c[0]
            term2 = c[1]
            term = c[0]+' '+c[1]
            if (term in sentence):
                #Found the sentence. Add only if not already in the summary
                if (sentence not in summary):
                    summary.append(sentence)
                    
    return collocations, summary
    '''
    print ("No. of sentences: "+str(len(summary)))
    index = 1
    for sentence in summary:
        print ("%s - %s" % (index,sentence))
        index+=1            
    '''

In [12]:
raw_corpus.values()

dict_values(['no i dont think so - we did this with oab and patients seem to be able to make correct decisions and understand their co-existent issues. in other words i do think that they are appropriate the symptoms mentioned only can be confuse with underactive bladder which may have similar type symptoms detrusor underactivity is a difficult to diagnose and not as common condition. in selected clinic and urodynamic series it may occur in 15% or so. i agree that the symptoms may partially overlap, but even md provider will likely give alpha blocker to those pts as they do not have a full uds examination to review. it\'s not inappropriate for these patients to take tamsulosin. the main issue is going to be whether or not the data are considered adequate to demonstrate that people who have not been diagnosed already with these comorbid conditions and take otc tamsulosin hydrochloride receive their diagnosis promptly. if the data are adequate to show that the otc drug does not delay the

In [13]:
dataset_clean.items()

dict_items([('Week1_Q1.txt', ['dont', 'think', 'patient', 'seem', 'able', 'make', 'correct', 'decision', 'understand', 'co-existent', 'issue', 'word', 'think', 'appropriate', 'symptom', 'mentioned', 'confuse', 'underactive', 'bladder', 'similar', 'type', 'symptom', 'detrusor', 'underactivity', 'difficult', 'diagnose', 'common', 'condition', 'selected', 'clinic', 'urodynamic', 'series', 'occur', 'agree', 'symptom', 'partially', 'overlap', 'even', 'provider', 'likely', 'give', 'alpha', 'blocker', 'full', 'examination', 'review', 'inappropriate', 'patient', 'take', 'tamsulosin', 'main', 'issue', 'going', 'whether', 'data', 'considered', 'adequate', 'demonstrate', 'people', 'diagnosed', 'already', 'comorbid', 'condition', 'take', 'tamsulosin', 'hydrochloride', 'receive', 'diagnosis', 'promptly', 'data', 'adequate', 'show', 'drug', 'delay', 'diagnosis', 'long', 'enough', 'result', 'adverse', 'consequence', 'acceptable', 'really', 'dont', 'think', 'guess', 'issue', 'underlying', 'comorbidity

In [14]:
clean_text2 = ""
for i in raw_corpus:
    clean_text2 = raw_corpus[i]

In [15]:
clean_text2

'looks pretty good... consider adding "heart failure" to causes (section 1) consider adding "heart disease" to ask your doctor section (3) i do not see the relationship between diabetes and "extreme difficulty passing even a drop of urine" - did you mean to put these together? thank you khal drogo for catching this typo. the last two bullets should be formatted as first level bullets, not sub-bullets. please see draft dfl & cil in resources on home page for correct formatting. i guess you\'re asking if we think the wording is reasonable. to ask if the label in the aus is adequate, we would need to see the data on the ss/aus study/ies and lcs. that said, under item 1, from a consumer standpoint the "if you think" phrase seems not quite on target. how would a consumer know within this warning whether or not they have these conditions, especially since the general term "urinary symptoms" is used. for warnings and action steps, more specificity is preferred, if possible. warning 2: seems o

In [16]:
collocation, summary = getSummary(clean_text2, 2)
print ('Collocations: ',collocation)
print()
print ("No. of sentences: "+str(len(summary)))
index = 1
for sentence in summary:
    print ("%s - %s" % (index,sentence))
    index+=1  

Collocations:  [('consider', 'adding'), ('adding', 'heart'), ('insert', 'list'), ('right', 'away'), ('scenario', 'question'), ('help', 'right'), ('liver', 'kidney'), ('urinate', 'severe'), ('also', 'caused'), ('kidney', 'disease'), ('medical', 'help'), ('pain', 'medical'), ('disease', 'talk'), ('told', 'doctor'), ('talk', 'doctor'), ('caused', 'infection'), ('retention', 'alert'), ('serious', 'condition'), ('urinary', 'retention'), ('symptom', 'also')]

No. of sentences: 14
1 - looks pretty good... consider adding "heart failure" to causes (section 1) consider adding "heart disease" to ask your doctor section (3) i do not see the relationship between diabetes and "extreme difficulty passing even a drop of urine" - did you mean to put these together?
2 - ...just to spur discussion, here's an attempt at what i mean: urinary symptoms can also be caused by: (insert list in warning 1) these conditions may be associated with symptoms such as: (insert list in warning 3) ...then put warning 2 

In [17]:
for i in raw_corpus:
    print()
    print('Summary of ',i)
    print()
    
    if i in ['Week1_Q2.txt']:
        min_count = 5
    elif i in ['Week2_Q3.txt']:
        min_count = 4
    elif i in ['Week2_Q1.txt']:
        min_count = 5
    else:
        min_count = 4
        
    collocation, summary = getSummary(raw_corpus[i], min_count)
    print ('Collocations: ',collocation)
    print()
    print ("No. of sentences: "+str(len(summary)))
    index = 1
    for sentence in summary:
        print ("%s - %s" % (index,sentence))
        index+=1  


Summary of  Week1_Q1.txt

Collocations:  [('alpha', 'blocker'), ('actual', 'study')]

No. of sentences: 5
1 - i agree that the symptoms may partially overlap, but even md provider will likely give alpha blocker to those pts as they do not have a full uds examination to review.
2 - i guess the issue is not the underlying comorbidity but whether the individual is taking medications for certain conditions that might interact with the alpha blocker.
3 - re brian stark: virtually all alpha blockers including tamsulosin have been studied with mostly all antihypertensive classes and the aes are not higher in those pts on antihypertensives.
4 - none of the currently mentioned conditions are an absolute contraindication to giving tamsulosin, and alpha blockers have been given regularly to patients with all of these co-morbidities.
5 - i will speak to that: alpha blockers for luts/bph are prescribed by pcp and uro and both groups prescribe probably equally common i would say the guidelines prov

In [18]:
cleantokens=[]
for i in dataset_clean.values():
    cleantokens+=i

print(cleantokens)
    
    


['dont', 'think', 'patient', 'seem', 'able', 'make', 'correct', 'decision', 'understand', 'co-existent', 'issue', 'word', 'think', 'appropriate', 'symptom', 'mentioned', 'confuse', 'underactive', 'bladder', 'similar', 'type', 'symptom', 'detrusor', 'underactivity', 'difficult', 'diagnose', 'common', 'condition', 'selected', 'clinic', 'urodynamic', 'series', 'occur', 'agree', 'symptom', 'partially', 'overlap', 'even', 'provider', 'likely', 'give', 'alpha', 'blocker', 'full', 'examination', 'review', 'inappropriate', 'patient', 'take', 'tamsulosin', 'main', 'issue', 'going', 'whether', 'data', 'considered', 'adequate', 'demonstrate', 'people', 'diagnosed', 'already', 'comorbid', 'condition', 'take', 'tamsulosin', 'hydrochloride', 'receive', 'diagnosis', 'promptly', 'data', 'adequate', 'show', 'drug', 'delay', 'diagnosis', 'long', 'enough', 'result', 'adverse', 'consequence', 'acceptable', 'really', 'dont', 'think', 'guess', 'issue', 'underlying', 'comorbidity', 'whether', 'individual', '

In [19]:
dataset_raw.values()

dict_values([['no i dont think so - we did this with oab and patients seem to be able to make correct decisions and understand their co-existent issues. in other words i do think that they are appropriate the symptoms mentioned only can be confuse with underactive bladder which may have similar type symptoms\n', 'detrusor underactivity is a difficult to diagnose and not as common condition. in selected clinic and urodynamic series it may occur in 15% or so. i agree that the symptoms may partially overlap, but even md provider will likely give alpha blocker to those pts as they do not have a full uds examination to review.\n', "it's not inappropriate for these patients to take tamsulosin. the main issue is going to be whether or not the data are considered adequate to demonstrate that people who have not been diagnosed already with these comorbid conditions and take otc tamsulosin hydrochloride receive their diagnosis promptly. if the data are adequate to show that the otc drug does not

In [20]:
path = "C:\\Users\\ziadm\\Downloads\\Web Data mining\\RAP\\Week1\\Week1_Q1.txt"

tokens_raw = nltk.word_tokenize(raw_corpus["Week1_Q1.txt"])

In [61]:
from nltk.corpus import wordnet
tokens = []

def get_lemma(token):
    #Return the morphological variant of this word
    lemma = wordnet.morphy(token)

    if lemma is None:
        return token
    else:
        return lemma

lemmas = [get_lemma(token) for token in tokens]

print (lemmas)

stop_words= set(nltk.corpus.stopwords.words('english'))
tokens_clean = [token for token in lemmas if (len(token)>4 and token not in stop_words)]

print(" CLEAN TOKENS")
print()
print (tokens_clean)

def clean_tokens(text):
    tokens_raw = nltk.word_tokenize(text)
    tokens = []
    for token in tokens_raw:
        if (token == " " or token.startswith('http') or token.startswith('@')):
            continue
        else:
            tokens.append(token)
    
    lemmas = [get_lemma(token) for token in tokens]
    tokens_clean = [token for token in lemmas if (len(token)>4 and token not in stop_words)]
    return tokens_clean

[]
 CLEAN TOKENS

[]


In [62]:
import re

tokens = [clean_tokens(token) for token in dataset3]

print(tokens)

[['think', 'patient', 'correct', 'decision', 'understand', 'co-existent', 'issue', 'words', 'think', 'appropriate', 'symptom', 'mention', 'confuse', 'underactive', 'bladder', 'similar', 'symptom'], ['detrusor', 'underactivity', 'difficult', 'diagnose', 'common', 'condition', 'select', 'clinic', 'urodynamic', 'series', 'occur', 'agree', 'symptom', 'partially', 'overlap', 'provider', 'likely', 'alpha', 'blocker', 'examination', 'review'], ['inappropriate', 'patient', 'tamsulosin', 'issue', 'going', 'whether', 'consider', 'adequate', 'demonstrate', 'people', 'diagnose', 'already', 'comorbid', 'conditions', 'tamsulosin', 'hydrochloride', 'receive', 'diagnosis', 'promptly', 'adequate', 'delay', 'diagnosis', 'enough', 'result', 'adverse', 'consequence', 'acceptable'], ['really', 'think', 'guess', 'issue', 'underlie', 'comorbidity', 'whether', 'individual', 'taking', 'medication', 'certain', 'conditions', 'might', 'interact', 'alpha', 'blocker', 'specifically', 'certain', 'anti-hypertensive',

In [63]:
from gensim import corpora
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(tweet) for tweet in tokens]

In [24]:
dictionary.token2id

{'appropriate': 0,
 'bladder': 1,
 'co-existent': 2,
 'confuse': 3,
 'correct': 4,
 'decision': 5,
 'issue': 6,
 'mention': 7,
 'patient': 8,
 'similar': 9,
 'symptom': 10,
 'think': 11,
 'underactive': 12,
 'understand': 13,
 'words': 14,
 'agree': 15,
 'alpha': 16,
 'blocker': 17,
 'clinic': 18,
 'common': 19,
 'condition': 20,
 'detrusor': 21,
 'diagnose': 22,
 'difficult': 23,
 'examination': 24,
 'likely': 25,
 'occur': 26,
 'overlap': 27,
 'partially': 28,
 'provider': 29,
 'review': 30,
 'select': 31,
 'series': 32,
 'underactivity': 33,
 'urodynamic': 34,
 'acceptable': 35,
 'adequate': 36,
 'adverse': 37,
 'already': 38,
 'comorbid': 39,
 'conditions': 40,
 'consequence': 41,
 'consider': 42,
 'delay': 43,
 'demonstrate': 44,
 'diagnosis': 45,
 'enough': 46,
 'going': 47,
 'hydrochloride': 48,
 'inappropriate': 49,
 'people': 50,
 'promptly': 51,
 'receive': 52,
 'result': 53,
 'tamsulosin': 54,
 'whether': 55,
 'anti-hypertensive': 56,
 'certain': 57,
 'comorbidity': 58,
 'gu

In [73]:
import gensim
k=4
iterations = 20
topic_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=k, id2word = dictionary, passes = iterations)
topics = topic_model.print_topics(num_words = 4)
for topic in topics:
    print (topic)

(0, '0.015*"symptom" + 0.014*"would" + 0.012*"dipstick" + 0.011*"study"')
(1, '0.021*"symptom" + 0.016*"conditions" + 0.016*"think" + 0.015*"would"')
(2, '0.014*"symptom" + 0.013*"label" + 0.011*"consumer" + 0.010*"would"')
(3, '0.024*"symptom" + 0.024*"prostate" + 0.016*"label" + 0.016*"cancer"')


In [71]:
import pyLDAvis.gensim
lda_vis = pyLDAvis.gensim.prepare(topic_model,corpus,dictionary,sort_topics=False)
pyLDAvis.display(lda_vis)

C:\Users\ziadm\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [27]:
print (cleantokens[0:100])

['dont', 'think', 'patient', 'seem', 'able', 'make', 'correct', 'decision', 'understand', 'co-existent', 'issue', 'word', 'think', 'appropriate', 'symptom', 'mentioned', 'confuse', 'underactive', 'bladder', 'similar', 'type', 'symptom', 'detrusor', 'underactivity', 'difficult', 'diagnose', 'common', 'condition', 'selected', 'clinic', 'urodynamic', 'series', 'occur', 'agree', 'symptom', 'partially', 'overlap', 'even', 'provider', 'likely', 'give', 'alpha', 'blocker', 'full', 'examination', 'review', 'inappropriate', 'patient', 'take', 'tamsulosin', 'main', 'issue', 'going', 'whether', 'data', 'considered', 'adequate', 'demonstrate', 'people', 'diagnosed', 'already', 'comorbid', 'condition', 'take', 'tamsulosin', 'hydrochloride', 'receive', 'diagnosis', 'promptly', 'data', 'adequate', 'show', 'drug', 'delay', 'diagnosis', 'long', 'enough', 'result', 'adverse', 'consequence', 'acceptable', 'really', 'dont', 'think', 'guess', 'issue', 'underlying', 'comorbidity', 'whether', 'individual', '

In [28]:
w2v_model = Word2Vec([cleantokens],size=32, sg=1, window = 5, min_count=3, seed = 20, workers=2)
print (len(w2v_model.wv.vocab))
print (w2v_model.wv.vocab)

520
{'dont': <gensim.models.keyedvectors.Vocab object at 0x0000023933247748>, 'think': <gensim.models.keyedvectors.Vocab object at 0x0000023933247CC0>, 'patient': <gensim.models.keyedvectors.Vocab object at 0x0000023933247A58>, 'seem': <gensim.models.keyedvectors.Vocab object at 0x0000023933247710>, 'able': <gensim.models.keyedvectors.Vocab object at 0x00000239332476D8>, 'make': <gensim.models.keyedvectors.Vocab object at 0x0000023933247860>, 'correct': <gensim.models.keyedvectors.Vocab object at 0x0000023933247E80>, 'decision': <gensim.models.keyedvectors.Vocab object at 0x0000023933247828>, 'understand': <gensim.models.keyedvectors.Vocab object at 0x0000023933247908>, 'issue': <gensim.models.keyedvectors.Vocab object at 0x0000023933247898>, 'word': <gensim.models.keyedvectors.Vocab object at 0x0000023933247CF8>, 'appropriate': <gensim.models.keyedvectors.Vocab object at 0x0000023933247A20>, 'symptom': <gensim.models.keyedvectors.Vocab object at 0x0000023933247C50>, 'mentioned': <gens

In [29]:
w2v_model.most_similar('patient')

C:\Users\ziadm\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('label', 0.9992484450340271),
 ('indication', 0.999201774597168),
 ('issue', 0.9990658760070801),
 ('week', 0.999046266078949),
 ('best', 0.9990084767341614),
 ('help', 0.9989648461341858),
 ('perhaps', 0.9989584684371948),
 ('year', 0.998920202255249),
 ('reason', 0.9989068508148193),
 ('without', 0.9989066123962402)]

In [30]:
#Retrieving the vocabulary from the 64-dimensional space
X_32D=w2v_model[w2v_model.wv.vocab]
# Transform the data and load up a Panda dataframe
tSNE = TSNE(n_components=2, n_iter=1000)
X_2D = tSNE.fit_transform(X_32D)
x2D_df = pd.DataFrame(X_2D, columns=['x','y'])
x2D_df['word'] = w2v_model.wv.vocab.keys()
x2D_df.head(10)

C:\Users\ziadm\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


,x,y,word
0,-5.797255,-16.492275,dont
1,-28.175179,13.525358,think
2,-29.958696,17.565172,patient
3,25.112700,26.041113,seem
4,-12.018383,-14.093454,able
5,-27.141981,12.263969,make
6,-14.604620,-13.416807,correct
7,-21.077454,-5.920253,decision
8,-19.028564,-8.264517,understand
9,-25.221865,3.626417,issue


In [31]:
# Configure the notebook to generate graph in a cell
# Always call this method before any visualization
output_notebook()

Loading BokehJS ...

In [32]:
# Extract a sample. If you have a powerful computer you can display all 17,000
plot = figure(plot_width=800, plot_height=800)
_ = plot.text(x=x2D_df.x, y=x2D_df.y, text=x2D_df.word)
show(plot)

In [33]:
print(w2v_model.most_similar(positive=['patient','risk','cancer']))

[('issue', 0.9995042085647583), ('well', 0.9993215203285217), ('agency', 0.9993035793304443), ('need', 0.999300479888916), ('year', 0.9992989301681519), ('reason', 0.9992505311965942), ('indication', 0.9992349147796631), ('many', 0.9992220401763916), ('treatment', 0.9992167949676514), ('week', 0.9992081522941589)]


C:\Users\ziadm\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [34]:
def apply_stemming(text):
    stemmer = nltk.PorterStemmer()
    normalized_text = [stemmer.stem(token) for token in text]
    return normalized_text



In [35]:
clean_tokens2= []
tokenized_token = []
index = 1
for token in dataset2:
    index+=1
    tokens = apply_stopwording(remove_punctuation(nltk.Text(nltk.word_tokenize(token))), 3)
    clean_text = apply_stemming(tokens)
    print ('[%s] - %s' % (index, clean_text))
    clean_tokens2.append(clean_text)
    tokenized_token.append(tokens)

[2] - ['echo', 'comment', 'risk', 'sampl', 'durat', 'event', 'occur', 'readili', 'explain', 'actual', 'circumst', 'subject', 'health', 'histori', 'event', 'often', 'case', 'sponsor', 'accept', 'small', 'risk', 'seriou', 'occur', 'possibl', 'drug', 'relat', 'readili', 'explain']
[3] - ['believ', 'rest', 'perhap', 'agenc', 'believ', 'dipstick', 'unnecessari', 'complic', 'process', 'limit', 'yield', 'call', 'medic', 'relev', 'condit']
[4] - ['tamsulosin', 'hydrochlorid', 'indic', 'tamsulosin', 'hydrochlorid', 'capsul', 'indic', 'treatment', 'sign', 'symptom', 'benign', 'prostat', 'hyperplasia', 'tamsulosin', 'hydrochlorid', 'capsul', 'indic', 'treatment', 'hypertens', 'discuss', 'point', 'group', 'might', 'focu', 'indic', 'phrase', 'consist', 'tamsulosin', 'hydrochlorid', 'support', 'limit', 'possibl']
[5] - ['otc', 'predic', 'label', 'suitabl', 'evid', 'label', 'comprehens', 'extent', 'provid', 'evid', 'self-select', 'consum', 'behav', 'context', 'label', 'drug', 'studi', 'coupl', 'comme

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
documents = [str((i)) for i in clean_tokens2]
#Tf-idf matrix


tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
print(documents)
print(tfidf_matrix)
features=tfidf_vectorizer.get_feature_names()
print(features)
print(tfidf_matrix.shape)

["['echo', 'comment', 'risk', 'sampl', 'durat', 'event', 'occur', 'readili', 'explain', 'actual', 'circumst', 'subject', 'health', 'histori', 'event', 'often', 'case', 'sponsor', 'accept', 'small', 'risk', 'seriou', 'occur', 'possibl', 'drug', 'relat', 'readili', 'explain']", "['believ', 'rest', 'perhap', 'agenc', 'believ', 'dipstick', 'unnecessari', 'complic', 'process', 'limit', 'yield', 'call', 'medic', 'relev', 'condit']", "['tamsulosin', 'hydrochlorid', 'indic', 'tamsulosin', 'hydrochlorid', 'capsul', 'indic', 'treatment', 'sign', 'symptom', 'benign', 'prostat', 'hyperplasia', 'tamsulosin', 'hydrochlorid', 'capsul', 'indic', 'treatment', 'hypertens', 'discuss', 'point', 'group', 'might', 'focu', 'indic', 'phrase', 'consist', 'tamsulosin', 'hydrochlorid', 'support', 'limit', 'possibl']", "['otc', 'predic', 'label', 'suitabl', 'evid', 'label', 'comprehens', 'extent', 'provid', 'evid', 'self-select', 'consum', 'behav', 'context', 'label', 'drug', 'studi', 'coupl', 'comment', 'daeneri

In [37]:
#co-sign similartiy between documents
from  sklearn.metrics.pairwise import cosine_similarity
sim_matrix = np.round(cosine_similarity(tfidf_matrix[0:7], tfidf_matrix),3)
print([' D1','D2','D3','D4','D5','D6','D7'])
print(sim_matrix)

[' D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7']
[[ 1.     0.     0.018 ...,  0.     0.     0.   ]
 [ 0.     1.     0.034 ...,  0.     0.349  0.039]
 [ 0.018  0.034  1.    ...,  0.152  0.042  0.011]
 ..., 
 [ 0.     0.146  0.    ...,  0.026  0.115  0.052]
 [ 0.     0.045  0.    ...,  0.037  0.181  0.   ]
 [ 0.011  0.032  0.2   ...,  0.195  0.03   0.085]]


In [38]:
set(dataset)

{'Week1_Q1.txt',
 'Week1_Q2.txt',
 'Week1_Q3.txt',
 'Week1_Q4.txt',
 'Week2_Q1.txt',
 'Week2_Q2.txt',
 'Week2_Q3.txt'}

In [39]:
lemmas_list=[]
token_list=[]

lemmas_list.extend(l for lemma in clean_tokens2 for l in lemma)
token_list.extend(t for token in tokenized_token for t in token)

token_dataframe = pd.DataFrame({'terms': token_list}, index = lemmas_list)
print(token_dataframe.head(10))

             terms
echo       echoing
comment   comments
risk          risk
sampl       sample
durat     duration
event        event
occur        occur
readili    readily
explain  explained
actual      actual


In [40]:
from sklearn.cluster import KMeans
k = 6
k_means = KMeans(n_clusters=k)
%time k_means.fit(tfidf_matrix)
clusters = k_means.labels_.tolist()
tokens_space = {'term':documents, 'cluster':clusters}
kmean_dataframe = pd.DataFrame(tokens_space,index=[clusters], columns =['term','cluster'])

Wall time: 178 ms


In [41]:
kmean_dataframe['cluster'].value_counts()

2    43
3    40
1    27
0    23
4    21
5    14
Name: cluster, dtype: int64

In [42]:
n=5

print ('Top %s terms within clusters' % n)
print()

sorted_centroids = k_means.cluster_centers_.argsort()[:, ::-1]
sent =""

for cluster_number in range(k):
    token_string = ''
    
    for ind in sorted_centroids[cluster_number, :n]:
        token_string = token_string + token_dataframe.loc[features[ind].split(' ')].values.tolist()[0][0] + ', '
    
    sent= sent + token_string
    print(token_string)
    print()
    print(type(token_string))
    print("Cluster %d: %s" % (cluster_number, token_string))


Top 5 terms within clusters

prostatic, symptoms, urinary, encouraged, reliever, 

<class 'str'>
Cluster 0: prostatic, symptoms, urinary, encouraged, reliever, 
dipstick, data, studies, think, comments, 

<class 'str'>
Cluster 1: dipstick, data, studies, think, comments, 
condition, patients, cancer, bladder, think, 

<class 'str'>
Cluster 2: condition, patients, cancer, bladder, think, 
event, death, would, drug, many, 

<class 'str'>
Cluster 3: event, death, would, drug, many, 
need, informational, symptoms, treatment, like, 

<class 'str'>
Cluster 4: need, informational, symptoms, treatment, like, 
self, selection, symptoms, purchase, point, 

<class 'str'>
Cluster 5: self, selection, symptoms, purchase, point, 


In [43]:
clean_text2

'looks pretty good... consider adding "heart failure" to causes (section 1) consider adding "heart disease" to ask your doctor section (3) i do not see the relationship between diabetes and "extreme difficulty passing even a drop of urine" - did you mean to put these together? thank you khal drogo for catching this typo. the last two bullets should be formatted as first level bullets, not sub-bullets. please see draft dfl & cil in resources on home page for correct formatting. i guess you\'re asking if we think the wording is reasonable. to ask if the label in the aus is adequate, we would need to see the data on the ss/aus study/ies and lcs. that said, under item 1, from a consumer standpoint the "if you think" phrase seems not quite on target. how would a consumer know within this warning whether or not they have these conditions, especially since the general term "urinary symptoms" is used. for warnings and action steps, more specificity is preferred, if possible. warning 2: seems o

In [44]:
#tokensnew = nltk.word_tokenize(clean_text2)
#cleaned_tokens2 = apply_stopwording(tokensnew,3)


for i in prefix: 
    tblog = TextBlob(str(dataset_raw[i]))
    print (tblog.sentiment)
    print (tblog.sentiment.polarity)
    print (tblog.sentiment.subjectivity)
    

Sentiment(polarity=0.13359126984126984, subjectivity=0.45107967032967033)
0.13359126984126984
0.45107967032967033
Sentiment(polarity=0.1357051622212912, subjectivity=0.439565907807843)
0.1357051622212912
0.439565907807843
Sentiment(polarity=0.05862391774891774, subjectivity=0.4658237734487735)
0.05862391774891774
0.4658237734487735
Sentiment(polarity=0.1295980669495521, subjectivity=0.47604314002828835)
0.1295980669495521
0.47604314002828835
Sentiment(polarity=0.0981788326615913, subjectivity=0.4689543215405286)
0.0981788326615913
0.4689543215405286
Sentiment(polarity=0.10206408267248969, subjectivity=0.469394930290948)
0.10206408267248969
0.469394930290948
Sentiment(polarity=0.1064038961038961, subjectivity=0.5041256854256854)
0.1064038961038961
0.5041256854256854


In [45]:
path = 'C:/Users/ziadm/Downloads/Web Data mining/'
def loadSentiWordNet(lfile):
    lf = open(lfile)
    lines=lf.readlines()
    lf.close
    lexicon = {}
    for line in lines:
        info = line.split("\t")
        try:
            p_score = float(info[2])
            n_score = float(info[3]) * -1.0
            words = info[4].split(" ")
            for word in words:
                if "#" in word:
                    term = word.split("#")
                    lexicon[term[0]]= p_score + n_score
        except:
            pass
    return lexicon

lexicon_dictionary = "newsenti.txt"
lex_fileName=path+lexicon_dictionary
lexicon_dictionary = loadSentiWordNet(lex_fileName)

In [46]:
lexicon_dictionary

{'able': 0.25,
 'unable': -0.125,
 'dorsal': 0.0,
 'abaxial': 0.0,
 'ventral': 0.0,
 'adaxial': 0.0,
 'acroscopic': 0.0,
 'basiscopic': 0.0,
 'abducting': 0.0,
 'abducent': 0.0,
 'adductive': 0.0,
 'adducting': 0.0,
 'adducent': 0.0,
 'nascent': 0.0,
 'emerging': 0.0,
 'emergent': -0.125,
 'dissilient': 0.25,
 'parturient': 0.0,
 'dying': -0.625,
 'moribund': -0.75,
 'last': 0.0,
 'abridged': 0.0,
 'shortened': 0.0,
 'cut': 0.0,
 'half-length': 0.375,
 'potted': 0.0,
 'unabridged': 0.5,
 'uncut': -0.125,
 'full-length': 0.25,
 'absolute': -0.25,
 'direct': 0.125,
 'unquestioning': -0.375,
 'implicit': 0.375,
 'infinite': 0.0,
 'living': 0.0,
 'relative': 0.0,
 'comparative': 0.0,
 'relational': 0.0,
 'absorptive': 0.0,
 'absorbent': 0.0,
 'sorbefacient': 0.375,
 'absorbefacient': 0.375,
 'assimilatory': -0.75,
 'assimilative': -0.75,
 'assimilating': -0.75,
 'hygroscopic': 0.0,
 'receptive': 0.25,
 'shock-absorbent': 0.0,
 'spongy': 0.125,
 'spongelike': 0.125,
 'thirsty': 0.25,
 'nona

In [47]:
s="uhuaffsasf"
if "#" in s:
    print("True")

In [48]:
testing=[]
tokensnew = nltk.word_tokenize(clean_text2)
cleaned_tokens2 = apply_stopwording(tokensnew,3)
testing.append(cleaned_tokens2)

In [49]:
for i in testing:
    print(len(i))
    for x in range(len(i)):
        print(x)

560
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

In [50]:
keys = lexicon_dictionary.keys()
print(lexicon_dictionary["concern"])


-0.525


In [51]:
score = 0.0
keys = lexicon_dictionary.keys()
for i in testing:
    for word in range(len(i)):
        if i[word] in keys:
            score = score + lexicon_dictionary[i[word]]
        
print (score)

-3.775


In [52]:
def scoreSentiment(testing,lex_dic):
    score = 0.0
    sentiment = "Neutral"
    keys = lex_dic.keys()
    for i in testing:
        for word in range(len(i)):
            if i[word] in keys:
                #print(i[word],lexicon_dictionary[i[word]])
                score = score + lexicon_dictionary[i[word]]   
    if score >1.5:
        sentiment = "Positive"
    elif score<-1.5:
        sentiment = "Negative"
    print(sentiment, score) 

scoreSentiment(testing,lexicon_dictionary)

Negative -3.775


In [53]:
for i in prefix: 
    dataset_raw[i]
    testing2=[]
    tokensnew2 = nltk.word_tokenize(str(dataset_raw[i]))
    cleaned_tokens3 = apply_stopwording(tokensnew2,3)
    testing2.append(cleaned_tokens3)
    print(i)
    scoreSentiment(testing2,lexicon_dictionary)

Week1_Q1.txt
Positive 9.724999999999998
Week1_Q2.txt
Negative -18.180999999999987
Week1_Q3.txt
Positive 6.2
Week1_Q4.txt
Neutral 0.825000000000001
Week2_Q1.txt
Negative -8.4
Week2_Q2.txt
Neutral 0.05000000000000071
Week2_Q3.txt
Negative -3.65
